In [1]:
!pip install spacy==2.3.0 --use-feature=2020-resolver
import spacy
from spacy.matcher import Matcher

     |████████████████████████████████| 10.0 MB 19.1 MB/s 
     |████████████████████████████████| 2.1 MB 72.7 MB/s 
     |████████████████████████████████| 69 kB 80.3 MB/s 
     |████████████████████████████████| 126 kB 95.7 MB/s 
     |████████████████████████████████| 3.7 MB 52.6 MB/s 
     |████████████████████████████████| 184 kB 74.8 MB/s 
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
import os
import json
import requests
import sys

In [3]:
FILE_PATH = "devoir/ift6285-le-devoir-articles/"
list_of_files  = os.listdir(FILE_PATH)

list_load_files = []

for file_ in list_of_files:

    json_file_path = FILE_PATH + file_

    with open(json_file_path, 'r') as j:
        contents = json.loads(j.read())

    list_load_files.append(contents)

In [4]:
text_by_articles = []

for i in range(0,len(list_load_files)):
    #for i in range(0,50):
    eleme = list_load_files[i]['BodyParagraphs']
    
    for j, part in enumerate(eleme):
        eleme[j] =  part.replace(u'\xa0', u' ').replace(u'\u2009', u' ')

    if(len(eleme) > 0):
        text_by_articles.append(" ".join(eleme))

In [5]:
len(text_by_articles)

10184

## Named entities

    ORG = Organization,companies,agencies, institutions, etc.
    PER = People, including fictional.
    LOC = Non-GPE locations, mountain ranges, bodies of water.
    MISC = Miscellaneous entities, e.g. events, nationalities, products or works of art

In [6]:
!spacy download fr_core_news_lg
!spacy download fr_core_news_sm
!spacy download fr_core_news_md

     |████████████████████████████████| 572.0 MB 53.1 MB/s 
  Created wheel for fr-core-news-lg: filename=fr_core_news_lg-2.3.0-py3-none-any.whl size=572090765 sha256=b011cbe1899a3d33c552e53bd3b4f9bc29ddc45b5844f8d0e489cdb812a9282f
  Stored in directory: /tmp/pip-ephem-wheel-cache-p_ktu284/wheels/27/63/e3/b10fab8e5f9855b3b7adb4356eb8c7df0254975048d9fef054
Successfully built fr-core-news-lg
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_lg')
     |████████████████████████████████| 14.7 MB 72.1 MB/s 
  Created wheel for fr-core-news-sm: filename=fr_core_news_sm-2.3.0-py3-none-any.whl size=14718366 sha256=374a5beb1d619bb4c3cc967e99170c46c82000839119e6af408b26ce13313455
  Stored in directory: /tmp/pip-ephem-wheel-cache-tovvgay_/wheels/cf/1e/f7/79485fa3afb8e3cb7490dedc3eb88b9fea2b4231a8209b77d1
Successfully built fr-core-news-sm
You should consid

In [7]:
import fr_core_news_lg
import fr_core_news_sm
import fr_core_news_md

nlp = fr_core_news_lg.load()
# nlp = fr_core_news_sm.load()
# nlp = fr_core_news_md.load()

In [8]:
def rule_per(avant2,avant1,cur,apres1,apres2):


    if (avant1.pos_ == cur.pos_ or avant1.pos_ == 'NOUN' or avant1.pos_ == 'PROPN' or str(avant1) =='-') and (apres1.pos_ == cur.pos_ or apres1.pos_ == 'NOUN' or apres1.pos_ == 'PROPN' or str(apres1) =='-') and (avant2.pos_ == cur.pos_ or avant2.pos_ == 'NOUN' or avant2.pos_ == 'PROPN') and (apres2.pos_ == cur.pos_ or apres2.pos_ == 'NOUN' or apres2.pos_ == 'PROPN'):
        return str(avant2)+" "+str(avant1)+" "+str(cur)+" "+str(apres1)+" "+str(apres2)

    if (avant1.pos_ == cur.pos_) and (apres1.pos_ == cur.pos_):
        return str(avant1)+" "+str(cur)+" "+str(apres1)

    if (avant1.pos_ == cur.pos_ or avant1.pos_ == 'NOUN' or avant1.pos_ == 'PROPN' or str(avant1) =='-') and (avant2.pos_ == cur.pos_ or avant2.pos_ == 'NOUN' or avant2.pos_ == 'PROPN'):
        return str(avant2)+" "+str(avant1)+" "+str(cur)

    if (apres1.pos_ == cur.pos_ or apres1.pos_ == 'NOUN' or apres1.pos_ == 'PROPN' or str(apres1) =='-') and (apres2.pos_ == cur.pos_ or apres2.pos_ == 'NOUN' or apres2.pos_ == 'PROPN'):
        return str(cur)+" "+str(apres1)+" "+str(apres2)
    
    if (avant1.pos_ == cur.pos_ or avant1.pos_ == 'NOUN' or avant1.pos_ == 'PROPN'):
        return str(avant1)+" "+str(cur)
    
    if (apres1.pos_ == cur.pos_ or apres1.pos_ == 'NOUN' or apres1.pos_ == 'PROPN'):
        return str(cur)+" "+str(apres1)


In [9]:
def check_label(label,doc):
    new = []
    for lab in label:
        if str(lab[1])=='MISC':
            cible=lab[0].split()[0]
            for i, word in enumerate(doc):
                if str(word) == cible :

                    if (doc[i].pos_ != 'PRON'and doc[i].pos_ != 'SCONJ' and doc[i].pos_ != 'PUNCT'):
                        new.append(lab)
        else:
            new.append(lab)

    return set(new)

In [10]:
def correction_spacy(label,doc_,text):
    label=set(label)
 
    label =check_label(label,doc_)

    all_add=[]
    for lab in label:
        cible=lab[0].split()[0]
        for i, word in enumerate(doc_):
            if str(word) == cible :

                if i >1 and i < (len(doc_)-2):
                    avant2 =doc_[i-2]
                    avant1 =doc_[i-1]
                    cur= doc_[i]
                    apres1 = doc_[i+1]
                    apres2 = doc_[i+2]

                    to_add =rule_per(avant2,avant1,cur,apres1,apres2)
                    
                    if cur.pos_ != 'DET' and to_add:
                        all_add.append((to_add,lab[1]))
                        to_add=None 
                   
    label_to_return = set(label.union(set(all_add)))
    return label_to_return


In [11]:
def print_all(list_to_print):
    for ele in list_to_print:
        print(ele)


In [ ]:

all = []
final=[]
for text in text_by_articles:
    doc = nlp(text)
    for ent in doc.ents:
        all.append((ent.text, ent.label_))
    # print(all)
    final.append(correction_spacy(all,doc,text))
    all=[]


print_all(final)


{('Union', 'LOC'), ('Fédération des médecins omnipraticiens du Québec', 'ORG'), ('psychologue Roger', 'PER'), ('Roger Godbout', 'PER'), ('sociologue Valérie', 'PER'), ('MAP', 'ORG'), ('Parlement européen', 'ORG'), ('Grecs', 'MISC'), ('Collège des médecins', 'ORG'), ('Université de Montréal', 'ORG'), ('ministre de la Justice', 'ORG'), ('Loi sur le temps légal', 'MISC'), ('États américains', 'LOC'), ('Le Devoir au Japon', 'MISC'), ('Conseil québécois des services éducatifs à la petite enfance', 'ORG'), ('Maghreb Agence Presse', 'ORG'), ('Hydro - Québec', 'LOC'), ('Québec', 'LOC'), ('Maroc', 'LOC'), ('Valérie Harvey', 'PER'), ('Maghreb Agence', 'ORG'), ('Assemblée nationale', 'ORG'), ('Hydro - Québec', 'ORG'), ('Commission européenne', 'ORG'), ('Département de psychiatrie', 'ORG'), ('Devoir', 'MISC'), ('Europe', 'LOC'), ('Louis-Olivier Batty', 'PER'), ('Parlement', 'ORG'), ('Commission', 'ORG'), ('Mme Harvey', 'PER'), ('Hydro', 'ORG'), ('Chypriotes', 'LOC'), ('Européens', 'LOC'), ('État',

Ensemble TEST

In [12]:
FILE_PATH = "test-projet2/"
list_of_files  = os.listdir(FILE_PATH)

list_load_files = []

for file_ in list_of_files:

    json_file_path = FILE_PATH + file_

    with open(json_file_path, 'r') as j:
        contents = json.loads(j.read())

    list_load_files.append(contents)

In [13]:
text_by_articles = []

for i in range(0,len(list_load_files)):
    #for i in range(0,50):
    eleme = list_load_files[i]['BodyParagraphs']
    
    for j, part in enumerate(eleme):
        eleme[j] =  part.replace(u'\xa0', u' ').replace(u'\u2009', u' ')

    if(len(eleme) > 0):
        text_by_articles.append(" ".join(eleme))

In [14]:
len(text_by_articles)

5

In [15]:
text_by_articles

["Ce texte fait partie d'un cahier spécial. Pendant la période estivale, ce sont surtout les touristes qui franchissent les portes du Musée des beaux-arts de Montréal, contrairement au reste de l’année où la clientèle montréalaise est à plus forte proportion. Pourtant, l’établissement a énormément à offrir aux Montréalais qui choisissent de le visiter seuls ou en famille. Visite guidée de cet établissement implanté au cœur de la métropole depuis 1860. Qui oserait franchir les portes d’un musée par une belle et ensoleillée journée d’été ? La compétition est forte entre un après-midi au parc et la visite d’une des galeries du Musée des beaux-arts de Montréal (MBAM), mais Pascale Chassé, directrice des communications au MBAM, ne s’en formalise pas trop : « Le beau temps nous amène quand même des visiteurs, surtout lorsqu’il y a des canicules et que les gens cherchent de l’air conditionné », lance-t-elle en riant. Et le MBAM a su bien exploiter son espace extérieur pour offrir un bain de s

In [42]:

all_test = []
final_test=[]
for text in text_by_articles:
    doc = nlp(text)
    for ent in doc.ents:
        all_test.append((ent.text, ent.label_))
    # print(all)
    # final_test.append(correction_spacy(all_test,doc,text))
    final_test.append(all_test)
    all_test=[]


print_all(final_test)


[('Musée des beaux', 'LOC'), ('Montréal', 'LOC'), ('Montréalais', 'LOC'), ('Musée des beaux', 'LOC'), ('Montréal', 'LOC'), ('MBAM', 'MISC'), ('Pascale Chassé', 'PER'), ('MBAM', 'MISC'), ('MBAM', 'MISC'), ('Le soleil', 'LOC'), ('Dale Chihuly', 'PER'), ('Musée', 'LOC'), ('Thierry Mugler', 'PER'), ('Couturissime', 'PER'), ('Jardin de sculptures', 'LOC'), ('Pascale Chassé', 'PER'), ('David Altmejd', 'PER'), ('Valérie Blass', 'PER'), ('Alexander Calder', 'PER'), ('Lynn Chadwick', 'PER'), ('Dale Chihuly', 'PER'), ('Jim Dine', 'PER'), ('Joe Fafard', 'PER'), ('Elisabeth Frink', 'PER'), ('Charles Joseph', 'PER'), ('Henry Moore', 'PER'), ('Jaume Plensa', 'PER'), ('Jean-Paul Riopelle', 'PER'), ('Auguste Rodin', 'PER'), ('Armand Vaillancourt', 'PER'), ('Thierry Mugler', 'PER'), ('Couturissime', 'PER'), ('MBAM', 'MISC'), ('Claire', 'PER'), ('Marc Bourgie', 'PER'), ('MBAM', 'MISC'), ('Omar Ba', 'PER'), ('Montréal', 'LOC'), ('Mme Chassé', 'PER'), ('Alanis Obomsawin', 'PER'), ('Office national du film